In [24]:
# ============================================================
# Cell 1 — LLM Adapter (Async + FairLLM Compatible)
# ============================================================

import os
import json
import asyncio
import httpx
from openai import AsyncOpenAI

# ==============================
# Configure your OpenAI API
# ==============================
# Paste your OpenAI key here
OPENAI_API_KEY = "notKey"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# ==============================
# Build Async OpenAI Client
# ==============================
# Disable HTTP/2 for stability on Windows / proxy networks
openai_http_client = httpx.AsyncClient(
    http2=False,
    timeout=httpx.Timeout(20.0, connect=20.0, read=20.0, write=20.0),
    trust_env=True,
    headers={"User-Agent": "fairllm-agent-weather/1.0"}
)

# Create AsyncOpenAI client
aclient = AsyncOpenAI(
    api_key=OPENAI_API_KEY,
    http_client=openai_http_client
)

# ============================================
# Define Async Adapter for FairLLM Planner
# ============================================
class OpenAIChatAdapter:
    """
    Async OpenAI adapter that works with FairLLM SimpleReActPlanner.
    Exposes the `ainvoke(messages)` method expected by the planner.
    """
    def __init__(self, model_name: str = "gpt-4.1-mini", temperature: float = 0.2):
        self.model_name = model_name
        self.temperature = temperature

    async def ainvoke(self, messages):
        # Convert FairLLM messages → OpenAI format
        oa_messages = [
            {
                "role": (m.get("role") if isinstance(m, dict) else getattr(m, "role", "user")),
                "content": (m.get("content") if isinstance(m, dict) else getattr(m, "content", "")),
            }
            for m in messages
        ]

        resp = await aclient.chat.completions.create(
            model=self.model_name,
            messages=oa_messages,
            temperature=self.temperature,
        )
        content = (resp.choices[0].message.content or "").strip()
        return type("LLMMessage", (), {"content": content})  # Planner expects .content

    def invoke(self, prompt: str) -> str:
        """Optional sync helper (not used by planner)."""
        async def _go():
            msg = await self.ainvoke([{"role": "user", "content": prompt}])
            return msg.content
        try:
            loop = asyncio.get_running_loop()
        except RuntimeError:
            return asyncio.run(_go())
        else:
            return loop.create_task(_go())

# ==============================
# Initialize and Test LLM
# ==============================
llm = OpenAIChatAdapter()

async def _ping():
    r = await llm.ainvoke([{"role": "user", "content": "Say OK"}])
    print("LLM ping:", r.content)

try:
    loop = asyncio.get_running_loop()
    loop.create_task(_ping())
except RuntimeError:
    asyncio.run(_ping())


LLM ping: OK


In [25]:
# Cell 2 — Tools: geocoding and weather, FairLLM-compatible (.use) with JSON-string handling

import json
import time
import httpx

class GeocodeCityTool:
    name = "geocode_city"
    description = "Geocode a city name to latitude and longitude using Open-Meteo Geocoding."
    input_schema = {
        "type": "object",
        "properties": {"city": {"type": "string"}},
        "required": ["city"]
    }

    def _parse_input(self, input_obj):
        # Accept raw string (e.g., "Denver") or {"city": "..."}; also handle JSON-string dicts
        if isinstance(input_obj, str):
            s = input_obj.strip()
            if s.startswith("{") and s.endswith("}"):
                try:
                    obj = json.loads(s)
                    return str(obj.get("city") or obj.get("name") or obj.get("q") or "").strip()
                except Exception:
                    pass
            return s
        if isinstance(input_obj, dict) and "city" in input_obj:
            return str(input_obj["city"]).strip()
        for k in ("name", "query", "q"):
            if isinstance(input_obj, dict) and k in input_obj:
                return str(input_obj[k]).strip()
        raise ValueError("geocode_city input must be a city name string or {'city': '...'}")

    def use(self, input_obj):
        city = self._parse_input(input_obj)
        params = {"name": city, "count": 1}
        with httpx.Client(timeout=10.0, headers={"User-Agent": "fairllm-agent-weather/1.0"}, http2=False) as cli:
            last_status = None
            for _ in range(3):
                r = cli.get("https://geocoding-api.open-meteo.com/v1/search", params=params)
                last_status = r.status_code
                if last_status == 200:
                    data = r.json()
                    results = data.get("results") or []
                    if results:
                        c0 = results[0]
                        out = {
                            "city": c0.get("name"),
                            "country": c0.get("country"),
                            "lat": c0.get("latitude"),
                            "lon": c0.get("longitude"),
                        }
                        return json.dumps(out)
                    return json.dumps({"error": f"City not found: {city}"})
                time.sleep(0.6)
        return json.dumps({"error": f"geocoding HTTP {last_status}", "city": city})


class WeatherAtLatLonTool:
    name = "weather_at_latlon"
    description = (
        "Retrieve 3 day hourly forecast from Open-Meteo for coordinates. "
        "Input may be {'lat': float, 'lon': float}, a JSON-string of that dict, or a string 'lat, lon'."
    )
    input_schema = {
        "type": "object",
        "properties": {"lat": {"type": "number"}, "lon": {"type": "number"}},
        "required": ["lat", "lon"]
    }

    BASE_URL = (
        "https://api.open-meteo.com/v1/forecast"
        "?latitude={lat}&longitude={lon}"
        "&hourly=temperature_2m,relative_humidity_2m,precipitation_probability,precipitation,"
        "cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,"
        "wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,"
        "wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,"
        "wind_gusts_10m,temperature_80m,temperature_120m,temperature_180m"
        "&forecast_days=3&wind_speed_unit=kn&temperature_unit=fahrenheit&precipitation_unit=inch"
    )

    def _parse_input(self, input_obj):
        # dict case
        if isinstance(input_obj, dict) and "lat" in input_obj and "lon" in input_obj:
            return float(input_obj["lat"]), float(input_obj["lon"])
        # JSON-string dict case
        if isinstance(input_obj, str):
            s = input_obj.strip()
            if s.startswith("{") and s.endswith("}"):
                try:
                    obj = json.loads(s)
                    if "lat" in obj and "lon" in obj:
                        return float(obj["lat"]), float(obj["lon"])
                except Exception:
                    pass
            # "lat, lon" case
            parts = s.replace(",", " ").split()
            nums = [p for p in parts if p.replace(".", "", 1).replace("-", "", 1).isdigit()]
            if len(nums) >= 2:
                return float(nums[0]), float(nums[1])
        # forgiving alternative keys
        if isinstance(input_obj, dict):
            for a, b in (("latitude", "longitude"), ("y", "x")):
                if a in input_obj and b in input_obj:
                    return float(input_obj[a]), float(input_obj[b])
        raise ValueError("weather_at_latlon input must be {'lat': ..., 'lon': ...}, a JSON-string of that dict, or 'lat, lon' string")

    def use(self, input_obj):
        lat, lon = self._parse_input(input_obj)
        lat = -90.0 if lat < -90.0 else 90.0 if lat > 90.0 else lat
        lon = ((lon + 180.0) % 360.0) - 180.0
        url = self.BASE_URL.format(lat=lat, lon=lon)

        with httpx.Client(timeout=12.0, headers={"User-Agent": "fairllm-agent-weather/1.0"}, http2=False) as cli:
            last_status = None
            for _ in range(3):
                r = cli.get(url)
                last_status = r.status_code
                if last_status == 200:
                    raw = r.json()
                    return json.dumps({
                        "source": "open-meteo",
                        "query": {"lat": lat, "lon": lon},
                        "hourly": raw.get("hourly", {}),
                        "meta": {
                            "elevation": raw.get("elevation"),
                            "timezone": raw.get("timezone"),
                            "tz_abbr": raw.get("timezone_abbreviation"),
                        }
                    })
                time.sleep(0.8)

        return json.dumps({"error": f"weather HTTP {last_status}", "lat": lat, "lon": lon})


In [26]:
# ============================================================
# Cell 3 — Agent wiring (mirrors math demo)
# ============================================================

from fairlib import (
    ToolRegistry,
    ToolExecutor,
    WorkingMemory,
    SimpleAgent,
    SimpleReActPlanner,
    RoleDefinition,
)

# Register tools
tool_registry = ToolRegistry()
tool_registry.register_tool(GeocodeCityTool())
tool_registry.register_tool(WeatherAtLatLonTool())

# Core agent components
executor = ToolExecutor(tool_registry)
memory = WorkingMemory()
planner = SimpleReActPlanner(llm, tool_registry)

# Strong role: force tool usage and require numeric outputs for the requested hour
planner.prompt_builder.role_definition = RoleDefinition(
    "You are a weather retrieval assistant. "
    "If the user provides a city name, first call the tool `geocode_city` to obtain lat and lon, "
    "then call the tool `weather_at_latlon` with those coordinates to fetch the forecast. "
    "If the user provides coordinates directly, call `weather_at_latlon` immediately. "
    "Do not invent numbers; always use the tools. "
    "Return exactly ONE JSON object as the final answer with keys: question, tool_calls, answer. "
    "In `answer`, include the specific hourly values for the time the user asked about. "
    "From the `weather_at_latlon` tool output, use `hourly.time` to locate the closest hour to the requested time "
    "(e.g., 'tomorrow at 2 PM'). If timezone is unclear, choose the closest matching hour string. "
    "Include, at minimum, these fields from the chosen hour: "
    "temperature_2m, relative_humidity_2m, wind_speed_10m, wind_direction_10m, wind_gusts_10m, "
    "precipitation_probability, precipitation. "
    "Also include `location` (lat, lon) and `chosen_time` (the ISO string from `hourly.time`) in `answer`."
)

# Optional: reinforce structured output
if hasattr(planner.prompt_builder, "output_format_hint"):
    planner.prompt_builder.output_format_hint = (
        "Final answer must be JSON with shape: "
        "{ 'question': str, "
        "  'tool_calls': [ { 'tool': str, 'input': object } ], "
        "  'answer': { "
        "     'location': { 'lat': number, 'lon': number }, "
        "     'chosen_time': str, "
        "     'values': { "
        "        'temperature_2m': number, "
        "        'relative_humidity_2m': number, "
        "        'wind_speed_10m': number, "
        "        'wind_direction_10m': number, "
        "        'wind_gusts_10m': number, "
        "        'precipitation_probability': number, "
        "        'precipitation': number "
        "     } "
        "  } "
        "}"
    )

# Assemble the agent
agent = SimpleAgent(
    llm=llm,
    planner=planner,
    tool_executor=executor,
    memory=memory,
    max_steps=6,
)


Registering tool: geocode_city
Registering tool: weather_at_latlon


In [27]:
# Cell 4 — Helper to run a prompt and pretty print JSON
def pretty_print(json_text: str):
    try:
        print(json.dumps(json.loads(json_text), indent=2))
    except Exception:
        print(json_text)

async def ask(q: str):
    print("USER:", q)
    resp = await agent.arun(q)
    pretty_print(resp)


In [28]:
# Cell 5 — The required test: Denver tomorrow at 2 PM
await ask("What will the weather be like in Denver tomorrow at 2 PM?")


USER: What will the weather be like in Denver tomorrow at 2 PM?
--- Step 1/6 ---
Thought: The user wants the weather forecast for Denver tomorrow at 2 PM. First, I need to get the latitude and longitude of Denver by using the geocode_city tool. Then I will use those coordinates to get the weather forecast for the specified time.
Action: Using tool 'geocode_city' with input 'Denver'
Observation: {"city": "Denver", "country": "United States", "lat": 39.73915, "lon": -104.9847}
--- Step 2/6 ---
Thought: I have the coordinates for Denver (lat: 39.73915, lon: -104.9847). The next step is to retrieve the weather forecast for Denver at those coordinates for tomorrow at 2 PM using the weather_at_latlon tool.
Action: Using tool 'weather_at_latlon' with input '{"lat": 39.73915, "lon": -104.9847}'
Observation: {"source": "open-meteo", "query": {"lat": 39.73915, "lon": -104.9847}, "hourly": {"time": ["2025-11-04T00:00", "2025-11-04T01:00", "2025-11-04T02:00", "2025-11-04T03:00", "2025-11-04T04:00"